In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：用于导出到云端的AutoML表格分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_tabular_classification_export_cloud.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_tabular_classification_export_cloud.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示如何使用Python的Vertex客户端库来创建表格分类模型，并将其导出为云模型，使用Google Cloud的AutoML。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[Iris数据集](https://www.tensorflow.org/datasets/catalog/iris)。 这个数据集不需要任何特征工程。 您在本教程中将使用的数据集版本存储在一个公共的云存储桶中。 训练好的模型可以预测来自三种不同种类的鸢尾花：山鸢尾、维吉尼亚鸢尾或者变色鸢尾的种类。

### 目标

在本教程中，您将使用 Vertex 客户端库从 Python 脚本创建一个 AutoML 表格分类模型，然后以 TensorFlow SavedModel 格式导出模型。您也可以选择使用 `gcloud` 命令行工具或在 Google Cloud 控制台上在线创建 AutoML 模型。

执行的步骤包括：

- 创建一个 Vertex `Dataset` 资源。
- 训练模型。
- 从 `Model` 资源中将模型导出到云存储。
- 在本地下载模型。
- 进行本地预测。

### 费用

本教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预期的使用量生成费用估算。

## 安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新版本的*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

安装了Vertex客户端库和Google *cloud-storage*后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果有选择的话，请确保在 GPU 运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您将获得 $300 的免费信用，可用于支付计算/存储成本。

2. [确保为您的项目启用了计费功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已经安装在谷歌云笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 将以 `!` 为前缀的行作为 shell 命令运行，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于本笔记本其余部分的操作。以下是支持Vertex的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不可以使用多区域存储桶用于Vertex训练。并非所有区域都支持所有的Vertex服务。有关各区域的最新支持信息，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加实时教程会话，您可能会使用一个共享的测试账户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并附加到将在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud账户

**如果您正在使用Google Cloud Notebook**，则您的环境已经过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并在提示进行身份验证时按照指示操作。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账号**。

在**服务账号名称**字段中，输入一个名称，然后点击**创建**。

在**授予此服务账号对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex”，并选择**Vertex管理员**。在过滤框中输入“存储对象管理员”，并选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中输入您的服务账号密钥的路径为GOOGLE_APPLICATION_CREDENTIALS变量，并运行单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

本教程旨在使用位于公共云存储桶中的训练数据，并使用本地云存储桶导出训练模型。您也可以使用您存储在本地云存储桶中的自有训练数据。

请在下面设置您的云存储桶的名称。桶的名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只要您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

##### 导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、管道和端点服务的顶点API服务端点。
- `PARENT`：数据集、模型、作业、管道和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML常量

设置独特于AutoML数据集和训练的常量：

- 数据集架构：告诉`Dataset`资源服务数据集的类型是什么。
- 数据标注（注释）架构：告诉`Dataset`资源服务数据是如何标记（注释）的。
- 数据集训练架构：告诉`Pipeline`资源服务训练模型的任务（例如，分类）。

In [ ]:
# Tabular Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/tables_1.0.0.yaml"
# Tabular Labeling type
LABEL_SCHEMA = (
    "gs://google-cloud-aiplatform/schema/dataset/ioformat/table_io_format_1.0.0.yaml"
)
# Tabular Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml"

### 安装Docker（Colab或本地）

如果您正在使用Google Cloud Notebook，则Docker已经安装好了。请跳过以下步骤。

在Colab上，默认情况下不安装Docker。如果您正在运行Colab，则需要按照以下步骤来安装Docker。

In [ ]:
if "google.colab" in sys.modules:
    ! sudo apt update
    ! sudo apt install apt-transport-https ca-certificates curl software-properties-common
    ! curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -
    ! sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"
    ! sudo apt update
    ! sudo apt install docker-ce

启动Docker服务

启动`docker`服务。

In [ ]:
if "google.colab" in sys.modules:
    ! sudo service docker start

# 教程

现在您已经准备好开始创建您自己的AutoML表格分类模型。

## 设置客户端

Vertex客户端库的工作方式是客户端/服务器模型。在您的端（Python脚本）中，您将创建一个客户端，用于向Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置好它们。

- 用于 `Dataset` 资源的 Dataset 服务。
- 用于 `Model` 资源的 Model 服务。
- 用于训练的 Pipeline 服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()

for client in clients.items():
    print(client)

数据集

现在您的客户端已经准备就绪，您的第一步是创建一个 `Dataset` 资源实例。这一步与 Vision、Video 和 Language 不同。对于那些产品，在创建 `Dataset` 资源之后，接着单独使用 `import_data` 方法导入数据。

对于表格数据，数据的导入推迟到训练管道开始训练模型时。我们要做什么不同呢？首先，您不会调用 `import_data` 方法。相反，当您创建数据集实例时，您会在 `Dataset` 资源的元数据中指定 CSV 文件的 Cloud Storage 位置或包含表格数据的 BigQuery 位置。

#### Cloud Storage

`metadata = {"input_config": {"gcs_source": {"uri": [gcs_uri]}}}`

Cloud Storage 路径的格式为：

    gs://[bucket_name]/[folder(s)/[file]

#### BigQuery

`metadata = {"input_config": {"bigquery_source": {"uri": [gcs_uri]}}}`

BigQuery 路径的格式为：

    bq://[collection].[dataset].[table]

请注意，`uri` 字段是一个列表，您可以在数据分布在多个文件中时输入多个 CSV 文件或 BigQuery 表格。

### 数据准备

对于表格数据，顶点`数据集`资源有一些要求。

- 必须在CSV文件或BigQuery查询中。

CSV

对于表格分类，CSV文件有一些要求：

- 第一行必须是表头 -- 请注意，这与Vision、Video和Language中的要求不同，那里要求没有标题。
- 除了一个列是特征之外，其他列都是特征。
- 一个列是标签，您将在随后创建训练管道时指定。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为Cloud Storage中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/tables/iris_1000.csv"

快速查看您的数据

您将使用存储在公共云存储桶中的鸢尾花数据集的一个版本，使用一个CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例数量，然后查看前几行。

您还需要知道头部标签列的列名，它被保存为`label_column`。对于这个数据集，它是CSV文件中的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

数据集

现在您的客户已经准备好，训练模型的第一步是创建一个托管数据集实例，然后将标记数据上传到其中。

### 创建`Dataset`资源实例

使用辅助函数`create_dataset`来创建`Dataset`资源的实例。此函数执行以下操作：

1. 使用数据集客户端服务。
2. 使用以下参数创建Vertex`Dataset`资源（`aip.Dataset`）：
 - `display_name`：您选择的可读名称。
 - `metadata_schema_uri`：数据集类型的模式。
 - `metadata`：标签数据的Cloud Storage或BigQuery位置。
3. 调用客户端数据集服务方法`create_dataset`，并使用以下参数：
 - `parent`：您的`Database`、`Model`和`Endpoint`资源的Vertex位置根路径。
 - `dataset`：您创建的Vertex数据集对象实例。
4. 该方法返回一个`operation`对象。

`operation`对象是Vertex处理长时间运行操作的异步调用方式。虽然这一步通常很快，但在项目中首次使用时，由于要进行配置，会有较长的延迟。

您可以使用`operation`对象来获取操作的状态（例如，创建`Dataset`资源）或取消操作，通过调用操作方法：

| 方法          | 描述            |
| ----------- | -----------    |
| result()    | 等待操作完成并以JSON格式返回结果对象。    |
| running()   | 返回操作是否仍在运行的True/False。       |
| done()      | 返回操作是否已完成的True/False。        |
| canceled()  | 返回操作是否已取消的True/False。      |
| cancel()    | 取消操作（可能需要多达30秒）。        |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, src_uri=None, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        if src_uri.startswith("gs://"):
            metadata = {"input_config": {"gcs_source": {"uri": [src_uri]}}}
        elif src_uri.startswith("bq://"):
            metadata = {"input_config": {"bigquery_source": {"uri": [src_uri]}}}
        dataset = aip.Dataset(
            display_name=name,
            metadata_schema_uri=schema,
            labels=labels,
            metadata=json_format.ParseDict(metadata, Value()),
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("iris-" + TIMESTAMP, DATA_SCHEMA, src_uri=IMPORT_FILE)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

训练模型

现在使用您的Vertex `Dataset`资源训练一个AutoML表格分类模型。要训练模型，请执行以下步骤：

1. 为`Dataset`资源创建一个Vertex训练管道。
2. 执行管道开始训练。

### 创建训练管道

您可能会问，我们何时使用管道？通常在工作（如训练）有多个步骤时，我们会按照顺序使用管道：执行步骤A，执行步骤B，依此类推。将这些步骤放入管道中，我们可以获得以下好处：

1. 可重用于后续训练工作。
2. 可以容器化并作为批处理作业运行。
3. 可以分布式运行。
4. 所有步骤都与相同的管道作业关联，以跟踪进度。

使用此辅助函数`create_pipeline`，它接受以下参数：

- `pipeline_name`：管道作业的可读名称。
- `model_name`：模型的可读名称。
- `dataset`：Vertex 完全限定的数据集标识符。
- `schema`：数据集标签（注释）训练模式。
- `task`：描述训练作业要求的字典。

辅助函数调用`Pipeline`客户端服务的方法`create_pipeline`，它接受以下参数：

- `parent`：您`数据集`，`模型`和`终端`资源的 Vertex 位置根路径。
- `training_pipeline`：管道训练作业的完整规范。

现在让我们更深入地研究构造`training_pipeline`规范的*最低*要求：

- `display_name`：管道作业的可读名称。
- `training_task_definition`：数据集标签（注释）训练模式。
- `training_task_inputs`：描述训练作业要求的字典。
- `model_to_upload`：模型的可读名称。
- `input_data_config`：数据集规范。
 - `dataset_id`：只有 Vertex 数据集标识符（非完全限定）-- 这是完全限定标识符的最后部分。
 - `fraction_split`：如果指定，数据集用于训练、测试和验证的百分比。否则，百分比会由自动机器学习自动选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与其他参数不同，`task`字段需要一个Google protobuf Struct，而不是一个Python（类似JSON）字典，它与Python字典非常相似。使用`json_format.ParseDict`方法进行转换。

您需要指定的最小字段为：

- `prediction_type`：我们是在进行“分类”还是“回归”。
- `target_column`：要预测的CSV标题列名称（即标签）。
- `train_budget_milli_node_hours`：用于训练模型的最大时间预算（计费时间），其中1000 = 1小时。
- `disable_early_stopping`：是否True/False，用于让AutoML自行判断是否提前停止训练或耗尽整个预算进行训练。
- `transformations`：指定每个特征列的特征工程。

对于`transformations`，列表必须为每列指定一个条目。外键字段指示相应列的特征工程类型。在本教程中，将其设置为`"auto"`，告诉AutoML自动确定。

最后，通过调用助手函数`create_pipeline`来创建流水线，该函数返回一个训练流水线对象的实例。

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "sepal_width"}},
    {"auto": {"column_name": "sepal_length"}},
    {"auto": {"column_name": "petal_length"}},
    {"auto": {"column_name": "petal_width"}},
]

In [ ]:
PIPE_NAME = "iris_pipe-" + TIMESTAMP
MODEL_NAME = "iris_model-" + TIMESTAMP

task = Value(
    struct_value=Struct(
        fields={
            "target_column": Value(string_value=label_column),
            "prediction_type": Value(string_value="classification"),
            "train_budget_milli_node_hours": Value(number_value=1000),
            "disable_early_stopping": Value(bool_value=False),
            "transformations": json_format.ParseDict(TRANSFORMATIONS, Value()),
        }
    )
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在，请保存您创建的训练管道的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取有关培训流程的信息

现在仅获取此培训流程实例的流程信息。通过调用作业客户端服务的`get_training_pipeline`方法，以下参数可以获得该作业的作业信息：

- `name`：Vertex完全限定的流程标识符。

当模型训练完成时，流程状态将为`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

部署

训练上述模型可能需要长达30分钟的时间。

一旦您的模型训练完成，您可以通过将 `end_time` 减去 `start_time` 来计算训练模型所需的实际时间。对于您的模型，您需要知道完全限定的顶点模型资源标识符，这是管道服务分配给它的。您可以从返回的管道实例中获取它，作为字段 `model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

## 模型信息

现在你的模型已经训练完成，你可以获取一些关于你的模型的信息。

评估模型资源

现在找出模型服务认为您的模型有多好。在训练过程中，数据集的一部分被设置为测试（保留）数据，这些数据被管道服务用于评估模型。

### 列出所有切片的评估

使用这个辅助函数`list_model_evaluations`，它接受以下参数：

- `name`：`Model`资源的Vertex完全合格的模型标识符。

这个辅助函数使用模型客户端服务的`list_model_evaluations`方法，该方法接受相同的参数。从调用中得到的响应对象是一个列表，其中每个元素是一个评估指标。

对于每个评估（您可能只有一个），然后打印评估中每个指标的所有关键名称，并对于一小部分（`logLoss`和`auPrc`），您将打印结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("logloss", metrics["logLoss"])
        print("auPrc", metrics["auPrc"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

## 导出为云模型

您可以将AutoML表格分类模型导出为TensorFlow SavedFormat模型，然后可以自定义部署到Google Cloud或者本地下载。使用此辅助函数`export_model`将模型导出到Google Cloud，该函数接受以下参数：

- `name`：`Model`资源的Vertex完全限定标识符。
- `gcs_dest`：用于存储SavedFormat模型文件的Cloud Storage位置。

此函数调用`Model`客户端服务的方法`export_model`，参数如下：

- `name`：`Model`资源的Vertex完全限定标识符。
- `output_config`：导出模型的目标信息。
  - `artifact_destination.output_uri_prefix`：用于存储SavedFormat模型文件的Cloud Storage位置。
  - `export_format_id`：将模型格式保存为的格式。对于AutoML表格分类，只有一种选项：
   - `tf-saved-model`：TensorFlow SavedFormat

该方法返回一个长时间运行的操作`response`。我们将通过调用`response.result()`同步等待操作完成，直到模型导出完成。

In [ ]:
MODEL_DIR = BUCKET_NAME + "/" + "iris"


def export_model(name, format, gcs_dest):
    output_config = {
        "artifact_destination": {"output_uri_prefix": gcs_dest},
        "export_format_id": format,
    }
    response = clients["model"].export_model(name=name, output_config=output_config)
    print("Long running operation:", response.operation.name)
    result = response.result(timeout=1800)
    metadata = response.operation.metadata
    artifact_uri = str(metadata.value).split("\\")[-1][4:-1]
    print("Artifact Uri", artifact_uri)
    return artifact_uri


model_package = export_model(model_to_deploy_id, "tf-saved-model", MODEL_DIR)

### 运行导出的模型包

您的模型现在以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中，同时还有一个模型服务器的二进制文件。从 Cloud Storage 存储桶中本地加载模型，然后您可以使用预构建的谷歌云容器在本地部署模型和服务二进制文件，用于运行模型服务器。

#### 模型包布局

让我们先看看导出的包中有什么。导出的模型包将存储在您的 Cloud Storage 存储桶中，路径为：

    <your-gcs-bucket>/<model_name>/model-<model id>/tf-saved-model/<timedstamp>

接下来，查看模型包的内容。

In [ ]:
print("Model Package:", model_package)
print("Contents:")
! gsutil ls $model_package

print("\nTF Saved Model")
path = model_package + "/predict"
files = ! gsutil ls $path
saved_dir = files[1]
print(saved_dir)
! gsutil ls $saved_dir

下载导出的模型包。

您的第一步是将导出的模型包下载到本地。

In [ ]:
! gsutil cp -r $model_package .

更改模型包目录名称

在本教程中，您将使用Docker启动模型包。包路径名称中最后一个组件的时间戳包含一个冒号“：”，这对Docker是无效的。您需要重新命名。在下一步中，我们将把最后一个组件重命名为“tbl_exported”。

In [ ]:
subdir = model_package.split("/")[-1]
print(subdir)
! mv $subdir tbl_exported

运行导出的模型包

此模型包依赖于一个预先构建的谷歌云容器，用于提供导出的 AutoML 表格分类模型：

"gcr.io/cloud-aiplatform/automl_tables/prediction_server"

目前 Vertex 终端不支持该容器。您可以在本地或另一台您管理的计算实例上启动模型服务器以及容器。

在本教程中，您将使用`docker`启动它。通过指定 `-d` 选项给 `docker`，Docker 容器将在此笔记本的后台运行。

In [ ]:
MODEL_SERVER = "gcr.io/cloud-aiplatform/automl_tables/prediction_server"
PORT = 8081

docker_id = ! docker run -d -v `pwd`/tbl_exported:/models/default -p 8081:8080 -it $MODEL_SERVER

检查健康状况

我们需要留一点时间等待模型加载。让我们暂停10秒，然后进行健康检查。如果健康检查返回为空，那么准备好了。

In [ ]:
import time

time.sleep(10)
! curl -X GET http://localhost:8081/health

#### 验证docker镜像的加载

您也可以使用`docker container ls`来验证Docker镜像的加载情况。我们将添加`--latest`来获取有关最新加载的信息（即此镜像）。

In [ ]:
! docker container ls --latest

## 发出预测请求

现在使用您导出的模型进行预测。

### 预测请求格式

预测请求的格式是一个 JSON 对象，如下所示：

    { "instances": [ { "column_name_1": value, "column_name_2": value, … } , … ] }

将您的预测请求放在一个文本文件中，如：

    test.json

In [ ]:
INSTANCES = {
    "instances": [
        {
            "petal_length": "1.4",
            "petal_width": "1.3",
            "sepal_length": "5.1",
            "sepal_width": "2.8",
        }
    ]
}

import json

with open("test.json", "w") as f:
    data = json.dumps(INSTANCES)
    f.write(data)

发送预测请求

您可以使用CURL发送预测请求：

curl -X POST --data @test.json http://localhost:8081/predict

In [ ]:
! curl -X POST --data @/home/jupyter/test.json http://localhost:8081/predict

关闭Docker容器

In [ ]:
print(docker_id)
! docker kill $docker_id

清理工作

要清理此项目中使用的所有GCP资源，您可以删除用于教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的单个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调优作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME